# 학습이 너무 오래걸려서 이것저것 건드려보다가 시간을 다 잡아먹어서
# 결국 3에폭만 학습한채 제출하게됬습니다 ^^;;;

# 데이터전처리

In [2]:
import pandas
import tensorflow
import matplotlib
from konlpy.tag import Mecab
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import re


def preprocess_sentences(kor_sentences, eng_sentences, num_words=10000):
    # 형태소 분석기 초기화
    mecab = Mecab()

    # 중복 데이터 제거를 위해 set을 이용
    # kor_sentences = list(set([re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ. ]', "", sentence) for sentence in kor_sentences]))
    # eng_sentences = list(set([re.sub(r'[^a-zA-Z. ]', "", sentence).lower() for sentence in eng_sentences]))
    
    # 데이터를 순회하며, 한국어와 영어 문장을 각각 정제합니다.
    cleaned_corpus = []
    for kor, eng in zip(kor_sentences, eng_sentences):
        kor = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ. ]', "", kor)
        eng = re.sub(r'[^a-zA-Z. ]', "", eng).lower()

        # 만약 둘 중 어느 한 문장이 비어 있다면 해당 문장 쌍은 제외합니다.
        if kor and eng:
            cleaned_corpus.append((kor, eng))

    # 토큰화
    kor_sentences, eng_sentences = zip(*cleaned_corpus)
    kor_sentences = [mecab.morphs(sentence) for sentence in kor_sentences]
    eng_sentences = [sentence.split() for sentence in eng_sentences]

    # 정수 인코딩
    kor_tokenizer = Tokenizer(num_words=num_words)
    kor_tokenizer.fit_on_texts(kor_sentences)
    kor_sequences = kor_tokenizer.texts_to_sequences(kor_sentences)

    eng_tokenizer = Tokenizer(num_words=num_words)
    eng_tokenizer.fit_on_texts(eng_sentences)
    eng_sequences = eng_tokenizer.texts_to_sequences(eng_sentences)

    # 시작 토큰과 종료 토큰 추가
    kor_sequences = [[1] + seq + [2] for seq in kor_sequences]
    eng_sequences = [[1] + seq + [2] for seq in eng_sequences]

    # 패딩
    kor_sequences = pad_sequences(kor_sequences, padding='post')
    eng_sequences = pad_sequences(eng_sequences, padding='post')

    return kor_sequences, eng_sequences, kor_tokenizer, eng_tokenizer, cleaned_corpus


# 데이터셋 준비

In [4]:
from sklearn.model_selection import train_test_split


한글파일경로 = "/aiffel/aiffel/korean-english-park.train.ko" #텍스트 파일
영어파일경로 = "/aiffel/aiffel/korean-english-park.train.en" #텍스트 파일

# 텍스트 파일 읽기
with open(한글파일경로, "r") as f:
    kor_sentences = f.readlines()

with open(영어파일경로, "r") as f:
    eng_sentences = f.readlines()

# 전처리 수행
kor_sequences, eng_sequences, kor_tokenizer, eng_tokenizer, cleaned_corpus = preprocess_sentences(kor_sentences, eng_sentences)

# 학습 데이터와 검증 데이터로 분리
kor_train, kor_val, eng_train, eng_val = train_test_split(kor_sequences, eng_sequences, test_size=0.1)

# 결과의 일부 출력
for i in range(5):
    print("Korean : ", ' '.join([kor_tokenizer.index_word[idx] for idx in kor_sequences[i] if idx != 0]))
    print("English : ", ' '.join([eng_tokenizer.index_word[idx] for idx in eng_sequences[i] if idx != 0]))
    print("\n")

Korean :  . 개인 용 컴퓨터 사용 의 상당 부분 은 이것 보다 수 있 느냐 이
English :  the much of personal computing is about can you top this to


Korean :  . 모든 광 마우스 와 마찬가지 로 이 광 마우스 도 책상 위 에 놓 는 마우스 를 필요 로 하 지 않 는다 . 이
English :  the so a mention a few weeks ago about a wireless mouse brought in another wireless to


Korean :  . 그러나 이것 은 또한 책상 도 필요 로 하 지 않 는다 . 이
English :  the like all mice but it also doesnt need a to


Korean :  . . 달러 하 는 이 최첨단 무선 광 마우스 는 에서 팔 그 외 에 어떤 부분 이 든 그 움직임 에 따라 의 움직임 을 조절 하 는 회전 운동 센서 를 사용 하 고 있 다 . 이
English :  the uses sensors to control the movement as you move your wrist arm whatever through the air. to


Korean :  . 정보 관리 들 은 동남 아시아 에서 의 선박 들 에 대한 많 은 테러 계획 들 이 실패 로 돌아갔 음 을 밝혔으며 세계 해상 교역 량 의 거의 분 의 을 운송 하 는 좁 은 해 로 인 해협 이 테러 공격 을 당하 기 쉽 다고 경고 하 고 있 다 . 이
English :  the intelligence officials have revealed a spate of foiled on ships in southeast asia and are warning that a narrow stretch of water carrying almost one third of the worlds maritime trade is vulnerable to a terr

In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam


BUFFER_SIZE = len(kor_sequences)
BATCH_SIZE = 32
steps_per_epoch = len(kor_sequences) // BATCH_SIZE
embedding_dim = 256
units = 512
kor_vocab_size = 10000
eng_vocab_size = 10000

# 학습 및 검증 데이터셋 준비
train_dataset = tf.data.Dataset.from_tensor_slices((kor_train, eng_train)).shuffle(BUFFER_SIZE)
train_dataset = train_dataset.batch(BATCH_SIZE, drop_remainder=True)

val_dataset = tf.data.Dataset.from_tensor_slices((kor_val, eng_val)).shuffle(BUFFER_SIZE)
val_dataset = val_dataset.batch(BATCH_SIZE, drop_remainder=True)

# 모델설계

In [8]:
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Masking
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Attention Layer
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, units):
        super(BahdanauAttention, self).__init__()
        self.W1 = Dense(units)
        self.W2 = Dense(units)
        self.V = Dense(1)

    def call(self, query, values):
        query_with_time_axis = tf.expand_dims(query, 1)
        score = self.V(tf.nn.tanh(self.W1(query_with_time_axis) + self.W2(values)))
        attention_weights = tf.nn.softmax(score, axis=1)
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis=1)
        return context_vector, attention_weights


class Encoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, enc_units):
        super(Encoder, self).__init__()
        self.enc_units = enc_units
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units,
                                       return_sequences=True,
                                       return_state=True)

    def call(self, x, hidden):
        x = self.embedding(x)
        output, state = self.gru(x, initial_state=hidden)
        return output, state

    def initialize_hidden_state(self, batch_sz):
        return tf.zeros((batch_sz, self.enc_units))


class Decoder(tf.keras.Model):
    def __init__(self, vocab_size, embedding_dim, dec_units):
        super(Decoder, self).__init__()
        self.dec_units = dec_units
        self.embedding = Embedding(vocab_size, embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units,
                                       return_sequences=True,
                                       return_state=True)
        self.fc = Dense(vocab_size)
        self.attention = BahdanauAttention(self.dec_units)

    def call(self, x, hidden, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        x = tf.concat([tf.expand_dims(context_vector, 1), x], axis=-1)
        output, state = self.gru(x)
        output = tf.reshape(output, (-1, output.shape[2]))
        x = self.fc(output)
        return x, state, attention_weights


# Parameters
BUFFER_SIZE = len(kor_sequences)
BATCH_SIZE = 32
steps_per_epoch = len(kor_sequences) // BATCH_SIZE
embedding_dim = 256
units = 512
kor_vocab_size = 10000
eng_vocab_size = 10000

# Encoder, Decoder 생성
encoder = Encoder(kor_vocab_size, embedding_dim, units)
decoder = Decoder(eng_vocab_size, embedding_dim, units)

# Optimizer, Loss 정의
optimizer = Adam()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True, reduction='none')

def loss_function(real, pred):
    mask = tf.math.logical_not(tf.math.equal(real, 0))
    loss_ = loss_object(real, pred)
    mask = tf.cast(mask, dtype=loss_.dtype)
    loss_ *= mask
    return tf.reduce_mean(loss_)

# Training
@tf.function
def train_step(inp, targ, enc_hidden):
    loss = 0

    with tf.GradientTape() as tape:
        enc_output, enc_hidden = encoder(inp, enc_hidden)
        dec_hidden = enc_hidden
        dec_input = tf.expand_dims([1] * BATCH_SIZE, 1)  # '<start>' is replaced with 1

        # Teacher forcing - feeding the target as the next input
        for t in range(1, targ.shape[1]):
            # passing enc_output to the decoder
            predictions, dec_hidden, _ = decoder(dec_input, dec_hidden, enc_output)
            loss += loss_function(targ[:, t], predictions)

            # using teacher forcing
            dec_input = tf.expand_dims(targ[:, t], 1)

    batch_loss = (loss / int(targ.shape[1]))
    variables = encoder.trainable_variables + decoder.trainable_variables
    gradients = tape.gradient(loss, variables)
    optimizer.apply_gradients(zip(gradients, variables))
    
    return batch_loss





# 학습

In [9]:
# 학습
EPOCHS = 3

for epoch in range(EPOCHS):
    print("#2")
    enc_hidden = encoder.initialize_hidden_state(BATCH_SIZE)
    total_loss = 0
    for (batch, (inp, targ)) in enumerate(train_dataset.take(steps_per_epoch)):
        batch_loss = train_step(inp, targ, enc_hidden)
        total_loss += batch_loss
    print(f'Epoch {epoch+1} Loss {total_loss/steps_per_epoch:.4f}')

    # 검증
    val_loss = 0
    val_steps_per_epoch = len(kor_val) // BATCH_SIZE
    for (batch, (inp, targ)) in enumerate(val_dataset.take(val_steps_per_epoch)):
        enc_hidden_val = encoder.initialize_hidden_state(BATCH_SIZE)
        enc_output_val, enc_hidden_val = encoder(inp, enc_hidden_val)
        dec_hidden_val = enc_hidden_val
        dec_input_val = tf.expand_dims([1] * BATCH_SIZE, 1)

        for t in range(1, targ.shape[1]):
            predictions, dec_hidden_val, _ = decoder(dec_input_val, dec_hidden_val, enc_output_val)
            val_loss += loss_function(targ[:, t], predictions)
            dec_input_val = tf.expand_dims(targ[:, t], 1)
    val_loss /= int(targ.shape[1])
    print(f'Validation Loss {val_loss/val_steps_per_epoch:.4f}')


#2
Epoch 1 Loss 1.2004
Validation Loss 1.2048
#2
Epoch 2 Loss 1.0156
Validation Loss 1.0907
#2
Epoch 3 Loss 0.9190
Validation Loss 1.0358


# 번역함수

In [13]:
def preprocess_and_encode(sentence, mecab, tokenizer, max_len):
    # Preprocess and tokenize
    sentence = re.sub(r'[^가-힣ㄱ-ㅎㅏ-ㅣ. ]', "", sentence)
    sentence = mecab.morphs(sentence)
    sentence = tokenizer.texts_to_sequences([sentence])

    # Add start and end token and pad sequence
    sentence = [[1] + seq + [2] for seq in sentence]
    sentence = pad_sequences(sentence, maxlen=max_len, padding='post')
    return sentence

def translate(sentence, encoder, decoder, mecab, kor_tokenizer, eng_tokenizer):
    # Preprocessing and encoding
    sentence = preprocess_and_encode(sentence, mecab, kor_tokenizer, kor_sequences.shape[-1])

    inputs = tf.convert_to_tensor(sentence)
    result = ''

    hidden = [tf.zeros((1, units))]
    enc_out, enc_hidden = encoder(inputs, hidden)

    dec_hidden = enc_hidden
    dec_input = tf.expand_dims([1], 0)

    for t in range(eng_sequences.shape[-1]):
        predictions, dec_hidden, attention_weights = decoder(dec_input,
                                                             dec_hidden,
                                                             enc_out)

        predicted_id = tf.argmax(predictions[0]).numpy()

        # Skip for start and end tokens
        if predicted_id == 1 or predicted_id == 2:
            continue

        result += eng_tokenizer.index_word[predicted_id] + ' '

        dec_input = tf.expand_dims([predicted_id], 0)

    return result.strip(), sentence


# 예문번역

In [14]:
# Testing the function
sentences = ["오바마는 대통령이다.", "시민들은 도시 속에 산다.", "커피는 필요 없다.", "일곱 명의 사망자가 발생했다."]

for sentence in sentences:
    print(f'Input: {sentence}')
    result, _ = translate(sentence, encoder, decoder, Mecab(), kor_tokenizer, eng_tokenizer)
    print(f'Translated: {result}\n')


Input: 오바마는 대통령이다.
Translated: obama is a lot of

Input: 시민들은 도시 속에 산다.
Translated: they are in

Input: 커피는 필요 없다.
Translated: is a lot of

Input: 일곱 명의 사망자가 발생했다.
Translated: death toll was killed in

